In [1]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, optimizers

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# Define the paths to your training and validation data
train_data_dir = '/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/train'
test_data_dir = '/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/test'

# Image dimensions and batch size
img_width, img_height = 224, 224
batch_size = 32

# Create an ImageDataGenerator for test data
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

# Create an ImageDataGenerator for test data
test_datagen = ImageDataGenerator(rescale=1.0 / 255.0)

# Create data generators
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)
val_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 4098 images belonging to 4 classes.
Found 1023 images belonging to 4 classes.
Found 1279 images belonging to 4 classes.


In [3]:
# Load the pre-trained ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

# Freeze the initial 5 layers
for layer in base_model.layers[:5]:
    layer.trainable = False

# Create a new model on top
model = models.Sequential()
model.add(base_model)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(4, activation='softmax'))

94765736/94765736 [==============================] - 0s 0us/step


In [7]:
model.compile(
    loss='categorical_crossentropy',
    optimizer=optimizers.RMSprop(learning_rate=1e-4),
    metrics=['accuracy']
)

# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=10,
    validation_data=val_generator,
    validation_steps=val_generator.samples // batch_size
)

model.save('Resnet50_Alzheimers.h5')

Epoch 1/10
128/128 [==============================] - 95s 537ms/step - loss: 0.1199 - accuracy: 0.9680 - val_loss: 195930.3281 - val_accuracy: 0.5040
Epoch 2/10
128/128 [==============================] - 66s 517ms/step - loss: 0.1357 - accuracy: 0.9749 - val_loss: 1505765.8750 - val_accuracy: 0.4980
Epoch 3/10
128/128 [==============================] - 64s 498ms/step - loss: 0.0918 - accuracy: 0.9732 - val_loss: 299237.7812 - val_accuracy: 0.4990
Epoch 4/10
128/128 [==============================] - 64s 495ms/step - loss: 0.1059 - accuracy: 0.9739 - val_loss: 468402.0000 - val_accuracy: 0.4990
Epoch 5/10
128/128 [==============================] - 65s 503ms/step - loss: 0.1192 - accuracy: 0.9774 - val_loss: 108472.8359 - val_accuracy: 0.4990
Epoch 6/10
128/128 [==============================] - 64s 500ms/step - loss: 0.0815 - accuracy: 0.9801 - val_loss: 562460.8750 - val_accuracy: 0.5020
Epoch 7/10
128/128 [==============================] - 64s 495ms/step - loss: 0.0750 - accuracy: 0.9

In [9]:
test_accuracy=model.evaluate(test_generator)
print(test_accuracy[1])


40/40 [==============================] - 4s 102ms/step - loss: 2965250.5000 - accuracy: 0.5004
0.5003909468650818
